### Imports

In [1]:
# pip install Augmentor scikit-learn gputil

In [2]:
import os
import re
import csv
import time
import GPUtil
import random
import shutil
import Augmentor
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Small, MobileNetV3Large, VGG16, InceptionV3, ResNet50, ResNet101, ResNet152
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mobilenet_v2
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as preprocess_input_mobilenet_v3
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inceptionv3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.resnet import preprocess_input as preprocess_input_resnet101
from tensorflow.keras.applications.resnet_v2 import preprocess_input as preprocess_input_resnet152

### GPU Check

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print(tf.test.is_built_with_cuda())

TensorFlow version: 2.8.0
True


In [4]:
# Get GPU information
gpus = GPUtil.getGPUs()
for gpu in gpus:
    print(f"GPU {gpu.id}: {gpu.name}, GPU Load: {gpu.load * 100}%")

GPU 0: NVIDIA GeForce RTX 2080 Ti, GPU Load: 0.0%
GPU 1: NVIDIA GeForce RTX 2080 Ti, GPU Load: 0.0%


### 8:1:1 split of each folder -> oversampling using augmentations -> creating csv files [all done only once]

In [ ]:
### Done only once. Hence, commented.

In [ ]:
# import os
# import shutil
# import random

# def split_data(input_folder, output_folder):
#     # Create train, val, and test folders
#     train_folder = os.path.join(output_folder, 'train')
#     val_folder = os.path.join(output_folder, 'val')
#     test_folder = os.path.join(output_folder, 'test')

#     os.makedirs(train_folder, exist_ok=True)
#     os.makedirs(val_folder, exist_ok=True)
#     os.makedirs(test_folder, exist_ok=True)

#     # Dictionary to store counts for each class
#     class_counts = {}

#     # Iterate through each class folder in the input folder
#     for class_folder in os.listdir(input_folder):
#         class_path = os.path.join(input_folder, class_folder)

#         # Skip if it's not a directory
#         if not os.path.isdir(class_path):
#             continue

#         # Create subfolders in train, val, and test
#         train_class_folder = os.path.join(train_folder, class_folder)
#         val_class_folder = os.path.join(val_folder, class_folder)
#         test_class_folder = os.path.join(test_folder, class_folder)

#         os.makedirs(train_class_folder, exist_ok=True)
#         os.makedirs(val_class_folder, exist_ok=True)
#         os.makedirs(test_class_folder, exist_ok=True)

#         # Get the list of images in the class folder
#         images = os.listdir(class_path)
#         random.shuffle(images)

#         # Calculate the number of images for each split
#         total_images = len(images)
#         train_split = int(0.6 * total_images)
#         val_split = int(0.2 * total_images)

#         # Copy images to train, val, and test folders
#         for i, image in enumerate(images):
#             src_path = os.path.join(class_path, image)
            
#             if i < train_split:
#                 dst_path = os.path.join(train_class_folder, image)
#             elif i < train_split + val_split:
#                 dst_path = os.path.join(val_class_folder, image)
#             else:
#                 dst_path = os.path.join(test_class_folder, image)

#             shutil.copy(src_path, dst_path)

#         # Update class counts dictionary
#         class_counts[class_folder] = {
#             'total': total_images,
#             'train': train_split,
#             'val': val_split,
#             'test': total_images - train_split - val_split
#         }

#     return class_counts


# # Provide the input and output folder paths
# input_folder_path = "/uoa/home/s04bs3/data/full/ALL/"
# output_folder_path = "/uoa/home/s04bs3/data/full/dasa/"

# counts = split_data(input_folder_path, output_folder_path)

# # Display counts for each class
# for class_name, count_info in counts.items():
#     print(f"Class: {class_name}")
#     print(f"Total: {count_info['total']} | Train: {count_info['train']} | Val: {count_info['val']} | Test: {count_info['test']}")
#     print("---")


In [ ]:
# main_folder = "/uoa/home/s04bs3/data/full/dasa/train"
# output_folder = "/uoa/home/s04bs3/data/full/dasa/train_o/"

# # Iterate over subfolders
# for subfolder in os.listdir(main_folder):
#     subfolder_path = os.path.join(main_folder, subfolder)

#     # Check if it's a directory
#     if os.path.isdir(subfolder_path):
#         # Check if the directory contains images
#         if any(f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')) for f in os.listdir(subfolder_path)):
#             # Create an Augmentor pipeline for each subfolder
#             pipeline = Augmentor.Pipeline(subfolder_path, output_directory=os.path.join(output_folder, subfolder))
#             # Add your augmentations here
#             pipeline.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
#             pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
#             pipeline.flip_left_right(probability=0.5)
#             pipeline.flip_top_bottom(probability=0.5)

#             # Execute the augmentation process
#             pipeline.sample(3000)  # Adjust the number of samples as needed
#         else:
#             print(f"No images found in {subfolder_path}. Skipping.")


In [9]:
# def create_csv(input_folder, output_csv):
#     # Open CSV file in write mode
#     with open(output_csv, 'w', newline='') as csv_file:
#         # Create CSV writer
#         csv_writer = csv.writer(csv_file)
        
#         # Write header
#         csv_writer.writerow(['Image_Path', 'Label'])

#         # Initialize count
#         total_rows = 0

#         # Iterate through each class folder in the input folder
#         for label in os.listdir(input_folder):
#             class_folder = os.path.join(input_folder, label)

#             # Skip if it's not a directory
#             if not os.path.isdir(class_folder):
#                 continue

#             # Iterate through each image in the class folder
#             for image in os.listdir(class_folder):
#                 # Get the image path
#                 image_path = os.path.join(class_folder, image)

#                 # Write the row to the CSV file
#                 csv_writer.writerow([image_path, label])

#                 # Increment the count
#                 total_rows += 1

#     return total_rows

# # Example usage:
# input_folder_path = "/uoa/home/s04bs3/data/full/dasa/train/"
# output_csv_path = "/uoa/home/s04bs3/data/full/dasa/train.csv"

# total_rows = create_csv(input_folder_path, output_csv_path)
# print(f'Total rows created: {total_rows}')


Total rows created: 12183


In [13]:
def check_duplicates(file):
    df = pd.read_csv(f'/uoa/home/s04bs3/data/full/dasa/{file}.csv')
    duplicates_specific_columnicates_specific_column = df[df.duplicated(subset=['Image_Path'])]
    print(f"\nDuplicates in {file} based on Image_Path:")
    print(duplicates_specific_column)

check_duplicates('train')
check_duplicates('val')
check_duplicates('test')


Duplicates in train based on Image_Path:
Empty DataFrame
Columns: [Image_Path, Label]
Index: []

Duplicates in val based on Image_Path:
Empty DataFrame
Columns: [Image_Path, Label]
Index: []

Duplicates in test based on Image_Path:
Empty DataFrame
Columns: [Image_Path, Label]
Index: []


In [28]:
import pandas as pd
import re

def extract_pattern(path):
    pattern = r'(\d{2}-\d{2}-\d{4}-\d{2}-\d{2}-\d{2}\.hvdfrm\d+\(\d+,\d+\)-Z\d+\.\d+)'
    match = re.search(pattern, path)
    return match.group() if match else None

def get_paths_with_pattern(df):
    return set(df['Image_Path'].apply(extract_pattern))

def check_leakage(train_path, val_path, test_path):
    train_df = pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)

    train_paths = get_paths_with_pattern(train_df)
    val_paths = get_paths_with_pattern(val_df)
    test_paths = get_paths_with_pattern(test_df)

    overlap_train_val = train_paths.intersection(val_paths)
    overlap_train_test = train_paths.intersection(test_paths)
    overlap_val_test = val_paths.intersection(test_paths)

    print("Overlap between train and val:", overlap_train_val)
    print("Overlap between train and test:", overlap_train_test)
    print("Overlap between val and test:", overlap_val_test)
    print("\n")

    return train_df, val_df, test_df, overlap_train_val, overlap_train_test, overlap_val_test

def remove_leakage(train_df, val_df, test_df, overlap_train_val, overlap_train_test, overlap_val_test):
    # Filter rows from train_df where Image_Path is in overlap_train_val or overlap_train_test
    train_df_no_overlap = train_df[~train_df['Image_Path'].apply(extract_pattern).isin(overlap_train_val.union(overlap_train_test))]

    # Filter rows from val_df where Image_Path is in overlap_val_test
    val_df_no_overlap = val_df[~val_df['Image_Path'].apply(extract_pattern).isin(overlap_val_test)]

    return train_df_no_overlap, val_df_no_overlap

def check_leakage_after_removal(train_df_no_overlap, val_df_no_overlap, test_path):
    test_df = pd.read_csv(test_path)

    train_paths_no_overlap = get_paths_with_pattern(train_df_no_overlap)
    val_paths_no_overlap = get_paths_with_pattern(val_df_no_overlap)
    test_paths = get_paths_with_pattern(test_df)

    overlap_train_val_no_overlap = train_paths_no_overlap.intersection(val_paths_no_overlap)
    overlap_train_test_no_overlap = train_paths_no_overlap.intersection(test_paths)
    overlap_val_test_no_overlap = val_paths_no_overlap.intersection(test_paths)

    print("Overlap between train (after removal) and val:", overlap_train_val_no_overlap)
    print("Overlap between train (after removal) and test:", overlap_train_test_no_overlap)
    print("Overlap between val (after removal) and test:", overlap_val_test_no_overlap)
    print("\n")

    return overlap_val_test_no_overlap

def write_to_new_csv(train_df_no_overlap, val_df_no_overlap):
    # Save the updated train_df and val_df without overlaps to new CSV files
#     train_df_no_overlap.to_csv('/uoa/home/s04bs3/data/full/dasa/train_no_overlap.csv', index=False)
#     val_df_no_overlap.to_csv('/uoa/home/s04bs3/data/full/dasa/val_no_overlap.csv', index=False)
    print(f"Filtered train set without overlaps saved to train_no_overlap.csv")
    print(f"Filtered val set without overlaps saved to val_no_overlap.csv")

# Example usage:
train_path = '/uoa/home/s04bs3/data/full/dasa/train.csv'
val_path = '/uoa/home/s04bs3/data/full/dasa/val.csv'
test_path = '/uoa/home/s04bs3/data/full/dasa/test.csv'

train_df, val_df, test_df, overlap_train_val, overlap_train_test, overlap_val_test = check_leakage(train_path, val_path, test_path)
train_df_no_overlap, val_df_no_overlap = remove_leakage(train_df, val_df, test_df, overlap_train_val, overlap_train_test, overlap_val_test)
overlap_val_test_no_overlap = check_leakage_after_removal(train_df_no_overlap, val_df_no_overlap, test_path)

# Verify that the overlap has been successfully removed
if not overlap_val_test_no_overlap:
    write_to_new_csv(train_df_no_overlap, val_df_no_overlap)
else:
    print("Error: Overlap still exists after removal. Please review the process.")

# Counts of overlaps
count_overlap_train_val = len(overlap_train_val)
count_overlap_train_test = len(overlap_train_test)
count_overlap_val_test = len(overlap_val_test)
print(f"\n\nCount of overlap between train and val: {count_overlap_train_val} -> removed from train.csv")
print(f"Count of overlap between train and test: {count_overlap_train_test} -> removed from train.csv")
print(f"Count of overlap between val and test: {count_overlap_val_test} -> removed from val.csv")


Overlap between train and val: {'21-09-2022-08-59-32.hvdfrm101(1814,323)-Z65.00', '21-09-2022-10-06-36.hvdfrm215(231,311)-Z109.50', None, '21-09-2022-10-13-08.hvdfrm484(113,1246)-Z111.00', '21-09-2022-09-59-02.hvdfrm289(247,122)-Z142.00', '21-09-2022-09-06-36.hvdfrm078(2213,1451)-Z76.00', '21-09-2022-09-04-35.hvdfrm454(1104,1483)-Z81.00', '21-09-2022-10-09-37.hvdfrm167(0,520)-Z129.00', '21-09-2022-10-12-08.hvdfrm373(1426,303)-Z89.50'}
Overlap between train and test: {'21-09-2022-10-03-04.hvdfrm281(1929,1236)-Z39.50', '21-09-2022-10-04-34.hvdfrm339(1475,18)-Z185.00', '21-09-2022-09-09-07.hvdfrm121(344,1343)-Z100.50', '21-09-2022-09-08-06.hvdfrm407(1241,353)-Z42.50', None, '21-09-2022-10-12-08.hvdfrm593(202,1883)-Z92.50', '21-09-2022-10-03-04.hvdfrm589(471,1632)-Z41.50'}
Overlap between val and test: {'21-09-2022-10-02-03.hvdfrm422(1058,317)-Z70.50', '21-09-2022-10-00-33.hvdfrm173(2184,752)-Z58.50', '21-09-2022-09-08-06.hvdfrm150(1989,495)-Z86.00', '21-09-2022-09-09-37.hvdfrm117(1438,965

### Functions to train, evaluate and save results

In [ ]:
def train_model(model_name, train_data, val_data, epochs=10, batch_size=32):
    
    target_size=(224,224)

    if model_name.lower() in ["mobilenetv2", "mobilenetv2-0.5", "mobilenetv2-0.75"]:
        prep_fn = preprocess_input_mobilenet_v2
    elif model_name.lower() == "vgg16":
        prep_fn = preprocess_input_vgg16
    elif model_name.lower() == "inceptionv3":
        prep_fn = preprocess_input_inceptionv3
        target_size=(299,299)
    elif model_name.lower() == "ResNet50":
        prep_fn = preprocess_input_resnet50
    elif model_name.lower() == "ResNet101":
        prep_fn = preprocess_input_resnet101
    elif model_name.lower() == "ResNet152":
        prep_fn = preprocess_input_resnet152
    else:
        prep_fn = preprocess_input_mobilenet_v3
    
    # Set up data generators
    train_datagen = ImageDataGenerator(preprocessing_function=prep_fn)
    val_datagen = ImageDataGenerator(preprocessing_function=prep_fn)

    # Set up data generators
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        x_col="Image_Path",
        y_col="Label",
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        x_col="Image_Path",
        y_col="Label",
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    # Build model based on model_name
    base_model = None

    if model_name == "MobileNetV2":
        base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "MobileNetV2-0.75":
        base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights="imagenet", alpha=0.75)
    elif model_name == "MobileNetV2-0.5":
        base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights="imagenet", alpha=0.5)
    elif model_name == "MobileNetV3Small":
        base_model = MobileNetV3Small(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "MobileNetV3Small-0.75":
        base_model = MobileNetV3Small(input_shape=(224,224, 3), include_top=False, weights="imagenet", alpha=0.75)
    elif model_name == "MobileNetV3Small-Min":
        base_model = MobileNetV3Small(input_shape=(224,224, 3), include_top=False, weights="imagenet", minimalistic=True)
    elif model_name == "MobileNetV3Large":
        base_model = MobileNetV3Large(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "MobileNetV3Large-0.75":
        base_model = MobileNetV3Large(input_shape=(224,224, 3), include_top=False, weights="imagenet", alpha=0.75)
    elif model_name == "MobileNetV3Large-Min":
        base_model = MobileNetV3Large(input_shape=(224,224, 3), include_top=False, weights="imagenet", minimalistic=True)
    elif model_name == "VGG16":
        base_model = VGG16(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "InceptionV3":
        base_model = InceptionV3(input_shape=(299,299, 3), include_top=False, weights="imagenet")
    elif model_name == "ResNet50":
        base_model = ResNet50(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "ResNet101":
        base_model = ResNet101(input_shape=(224,224, 3), include_top=False, weights="imagenet")
    elif model_name == "ResNet152":
        base_model = ResNet152(input_shape=(224,224, 3), include_top=False, weights="imagenet")

    if base_model:
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(12, activation='softmax')
        ])

        model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

        # Define callbacks (EarlyStopping)
        early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint(f"/uoa/home/s04bs3/data/full/dasa/models/{model_name}.h5", save_best_only=True)

        # Start the timer for training
        start_time = time.time()

        # Train the model
        history = model.fit(
            train_generator,
            epochs=epochs,
            validation_data=val_generator,
            callbacks=[early_stopping, model_checkpoint]
        )

        # Stop the timer for training
        end_time = time.time()

        # Calculate and print the training time
        training_time = round(end_time - start_time,2)
        print(f"Training Time: {training_time} seconds")

        return model, history

    else:
        print(f"Model {model_name} not recognized.")
        return None, None

##############################################################################################################

def evaluate_model(model_name, model_path, test_csv_path, target_size=(224, 224), batch_size=32):
    # Load the test data
    test_data = pd.read_csv(test_csv_path)

    # Load the saved model
    model = load_model(model_path)

    # Create an ImageDataGenerator for test data
    if model_name.lower() in ["mobilenetv2", "mobilenetv2-0.5", "mobilenetv2-0.75"]:
        prep_fn = preprocess_input_mobilenet_v2
    elif model_name.lower() == "vgg16":
        prep_fn = preprocess_input_vgg16
    elif model_name.lower() == "inceptionv3":
        prep_fn = preprocess_input_inceptionv3
        target_size = (299,299)
    elif model_name.lower() == "ResNet50":
        prep_fn = preprocess_input_resnet50
    elif model_name.lower() == "ResNet101":
        prep_fn = preprocess_input_resnet101
    elif model_name.lower() == "ResNet152":
        prep_fn = preprocess_input_resnet152
    else:
        prep_fn = preprocess_input_mobilenet_v3
    test_datagen = ImageDataGenerator(preprocessing_function=prep_fn)  # You may need to adjust this based on your training data preprocessing

    # Configure the test generator
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_data,
        x_col="Image_Path",
        y_col="Label",
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False  # Important: set shuffle to False for reproducibility
    )

    # Start the timer for testing
    start_time = time.time()

    # Get predictions
    y_pred = model.predict(test_generator)

    # Stop the timer for testing
    end_time = time.time()

    # Calculate and print the testing time
    testing_time = round(end_time - start_time, 2)
    print(f"Testing Time: {testing_time} seconds")

    # Convert predictions to class labels
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Convert true labels to class labels
    y_true = test_generator.classes

    # Calculate and print confusion matrix
    conf_mat = confusion_matrix(y_true, y_pred_classes)
    print("Confusion Matrix:")
    print(conf_mat)

    # Calculate and print classification report
    class_labels = list(test_generator.class_indices.keys())
    report = classification_report(y_true, y_pred_classes, target_names=class_labels)
    print("\nClassification Report:")
    print(report)

    save_results(model_name, conf_mat, report, testing_time)

##############################################################################################################

def save_results(model_name, conf_mat, report, testing_time):
    # Save all results in a single text file
    with open(f"/uoa/home/s04bs3/data/full/dasa/results/{model_name}_results.txt", 'w') as file:
        file.write("Confusion Matrix:\n")
        file.write(np.array_str(conf_mat))
        file.write("\n\nClassification Report:\n")
        file.write(report)
        file.write(f"\nTesting Time: {testing_time} seconds")

### Train & Test

In [ ]:
model_name = ["MobileNetV2-0.5","MobileNetV2-0.75", "MobileNetV2-0.5",
              "MobileNetV3Small", "MobileNetV3Small-0.75", "MobileNetV3Small-Min",
              "MobileNetV3Large", "MobileNetV3Large-0.75", "MobileNetV3Large-Min",
              "VGG16","InceptionV3", "ResNet50", "ResNet101","ResNet152"]

train_data = pd.read_csv("/uoa/home/s04bs3/data/full/dasa/train_o.csv")
val_data = pd.read_csv("/uoa/home/s04bs3/data/full/dasa/val.csv")
test_csv_path = '/uoa/home/s04bs3/data/full/dasa/test.csv'

for n in range(len(model_name)):
    # Train the model
    model, history = train_model(model_name[n], train_data, val_data, epochs=30, batch_size=32)

    # Evaluate the model
    model_path = f'/uoa/home/s04bs3/data/full/dasa/models/{model_name[n]}.h5'    
    evaluate_model(model_name[n], model_path, test_csv_path)


In [ ]:
# Replace this with the path to your saved models directory
models_directory = '/uoa/home/s04bs3/data/full/dasa/models/'

# List of model names
model_name = ["MobileNetV2", "MobileNetV2-0.75", "MobileNetV2-0.5",
              "MobileNetV3Small", "MobileNetV3Small-0.75", "MobileNetV3Small-Min",
              "MobileNetV3Large", "MobileNetV3Large-0.75", "MobileNetV3Large-Min",
              "VGG16","InceptionV3", "ResNet50", "ResNet101","ResNet152"]

for model in model_name:
    # Load the model
    model_path = os.path.join(models_directory, f'{model}.h5')
    loaded_model = load_model(model_path)

    # Get the size of the model file
    model_size_bytes = os.path.getsize(model_path)

    # Convert size to MB
    model_size_MB = model_size_bytes / (1024 * 1024)

    # Get the number of parameters (in millions)
    num_params = loaded_model.count_params() / 1e6

    print(f"{model} - Model Size: {model_size_MB:.2f} MB | Number of Parameters: {num_params:.2f} million")
